In [38]:
import pandas as pd
import numpy as np
import datetime

import pymongo
from pymongo import MongoClient

from skyfield.api import load
from skyfield.framelib import ecliptic_frame

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.metrics import mean_squared_error

import matplotlib.pyplot as plt
%matplotlib inline

In [39]:
client = MongoClient()
db = client.get_database('binanceHistoricCandles')
mycollection = db['BTCUSDT1w']
document = mycollection.find_one({'pair': 'BTCUSDT', 'time': '1w'})
candles = document['candles']

In [40]:
# Just for reference
#  [
#   [
#     1499040000000,      // Open time
#     "0.01634790",       // Open
#     "0.80000000",       // High
#     "0.01575800",       // Low
#     "0.01577100",       // Close
#     "148976.11427815",  // Volume
#     1499644799999,      // Close time
#     "2434.19055334",    // Quote asset volume
#     308,                // Number of trades
#     "1756.87402397",    // Taker buy base asset volume
#     "28.46694368",      // Taker buy quote asset volume
#     "17928899.62484339" // Ignore.
#   ]
# ]

Lets take only the colunms with whatever we consider important indicators like

- Open Value
- High
- Low
- Close
- Volume
- Taker buy base asset volume, because is the volume involved in direct market price buys (is a bullish indicator)

In [41]:
columns = ['Open time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Close time','Quote asset volume', 'Number of trades', 'Taker buy base asset volume', 'Taker buy quote asset volume', 'Ignore']
df = pd.DataFrame(candles, columns=columns)
df = df.drop(['Quote asset volume', 'Number of trades', 'Taker buy quote asset volume', 'Ignore'], axis=1)
df.head()

,Open time,Open,High,Low,Close,Volume,Close time,Taker buy base asset volume
0,1502668800000,4261.48000000,4485.39000000,3850.00000000,4086.29000000,2843.43142600,1503273599999,2240.24924000
1,1503273600000,4069.13000000,4453.91000000,3400.00000000,4310.01000000,4599.39662900,1503878399999,1755.53026500
2,1503878400000,4310.01000000,4939.19000000,4124.54000000,4509.08000000,4753.84337600,1504483199999,1371.03442800
3,1504483200000,4505.00000000,4788.59000000,3603.00000000,4130.37000000,6382.78774500,1505087999999,2821.42273900
4,1505088000000,4153.62000000,4394.59000000,2817.00000000,3699.99000000,8106.70512700,1505692799999,3763.12845200


In [42]:
round_to_2dp = lambda x: round(float(x), 2)
df['Open']      = df['Open'].apply(round_to_2dp)
df['High']      = df['High'].apply(round_to_2dp)
df['Low']       = df['Low'].apply(round_to_2dp)
df['Close']     = df['Close'].apply(round_to_2dp)
df.head()

,Open time,Open,High,Low,Close,Volume,Close time,Taker buy base asset volume
0,1502668800000,4261.48,4485.39,3850.00,4086.29,2843.43142600,1503273599999,2240.24924000
1,1503273600000,4069.13,4453.91,3400.00,4310.01,4599.39662900,1503878399999,1755.53026500
2,1503878400000,4310.01,4939.19,4124.54,4509.08,4753.84337600,1504483199999,1371.03442800
3,1504483200000,4505.00,4788.59,3603.00,4130.37,6382.78774500,1505087999999,2821.42273900
4,1505088000000,4153.62,4394.59,2817.00,3699.99,8106.70512700,1505692799999,3763.12845200


In [43]:
# define a lambda function to convert milliseconds to date in dd/mm/yyyy format
date_to_string = lambda x: datetime.datetime.fromtimestamp(x/1000.0).strftime('%Y/%m/%d %H:%M')

# apply the lambda function to the 'B' column
df['Open time'] = df['Open time'].apply(date_to_string)
df['Close time'] = df['Close time'].apply(date_to_string)

df.head()

,Open time,Open,High,Low,Close,Volume,Close time,Taker buy base asset volume
0,2017/08/14 02:00,4261.48,4485.39,3850.00,4086.29,2843.43142600,2017/08/21 01:59,2240.24924000
1,2017/08/21 02:00,4069.13,4453.91,3400.00,4310.01,4599.39662900,2017/08/28 01:59,1755.53026500
2,2017/08/28 02:00,4310.01,4939.19,4124.54,4509.08,4753.84337600,2017/09/04 01:59,1371.03442800
3,2017/09/04 02:00,4505.00,4788.59,3603.00,4130.37,6382.78774500,2017/09/11 01:59,2821.42273900
4,2017/09/11 02:00,4153.62,4394.59,2817.00,3699.99,8106.70512700,2017/09/18 01:59,3763.12845200


In [44]:
def get_moon_phase_by_degrees(dateString):
    """Returns a tuple that describes the moon phase in moon degrees, percentage of visible moon, and a string describing the phase"""
    dt = datetime.datetime.strptime(dateString, '%Y/%m/%d %H:%M')
    year = int(dt.year)
    month = int(dt.month)
    day = int(dt.day)
    hour = int(dt.hour)
    minute = int(dt.minute)
    ts = load.timescale()
    t = ts.utc(year, month, day, hour, minute)

    eph = load('de421.bsp')
    sun, moon, earth = eph['sun'], eph['moon'], eph['earth']

    e = earth.at(t)
    s = e.observe(sun).apparent()
    m = e.observe(moon).apparent()

    _, slon, _ = s.frame_latlon(ecliptic_frame)
    _, mlon, _ = m.frame_latlon(ecliptic_frame)

    phase = (mlon.degrees - slon.degrees) % 360.0
    percent = 100.0 * m.fraction_illuminated(sun)
    
    value = int(phase)
    # "new moon", "waxing crescent", "first quarter", "waxing gibbous", "full moon", "waning gibbous", "last quarter", or "waning crescent".
    # 0 degrees represents a new moon, 90 degrees represents a first quarter moon, 
    # 180 degrees represents a full moon, and 270 degrees represents a last quarter moon.
    ranges = {
        tuple(range(0, 45)): 'New Moon',
        tuple(range(45, 90)): 'Waxing Crescent',
        tuple(range(90, 135)): 'First Quarter',
        tuple(range(135, 180)): 'Waxing Gibbous',
        tuple(range(180, 255)): 'Full Moon',
        tuple(range(225, 270)): 'Waning Gibbous',
        tuple(range(270, 315)): 'Last Quarter',
        tuple(range(315, 359)): 'Waning Crescent',
        tuple(range(359, 360)): 'New Moon'
    }
    for r, message in ranges.items():
        if value in r:
            return round(phase,2), round(percent,2), message 
            continue
            print('Value is outside the range')
    return None


### Let's Get the moon phase on open time and close time

- Open Moon is the moon phase on the open time
- Close Moon is the moon phase on the close time
- Each moon is represented in a tuple of the form (degrees, percentage of visible moon, and a string definig the phase)

In [45]:
df['Open Moon'] = df['Open time'].apply(get_moon_phase_by_degrees)
df['Close Moon'] = df['Close time'].apply(get_moon_phase_by_degrees)
df.head()

,Open time,Open,High,Low,Close,Volume,Close time,Taker buy base asset volume,Open Moon,Close Moon
0,2017/08/14 02:00,4261.48,4485.39,3850.00,4086.29,2843.43142600,2017/08/21 01:59,2240.24924000,"(257.36, 61.02, Waning Gibbous)","(350.94, 0.63, Waning Crescent)"
1,2017/08/21 02:00,4069.13,4453.91,3400.00,4310.01,4599.39662900,2017/08/28 01:59,1755.53026500,"(350.95, 0.63, Waning Crescent)","(76.09, 38.15, Waxing Crescent)"
2,2017/08/28 02:00,4310.01,4939.19,4124.54,4509.08,4753.84337600,2017/09/04 01:59,1371.03442800,"(76.1, 38.16, Waxing Crescent)","(153.7, 94.84, Waxing Gibbous)"
3,2017/09/04 02:00,4505.00,4788.59,3603.00,4130.37,6382.78774500,2017/09/11 01:59,2821.42273900,"(153.7, 94.84, Waxing Gibbous)","(241.14, 74.13, Full Moon)"
4,2017/09/11 02:00,4153.62,4394.59,2817.00,3699.99,8106.70512700,2017/09/18 01:59,3763.12845200,"(241.15, 74.12, Full Moon)","(332.99, 5.48, Waning Crescent)"


In [46]:
df.sample()

,Open time,Open,High,Low,Close,Volume,Close time,Taker buy base asset volume,Open Moon,Close Moon
110,2019/09/23 02:00,10028.05,10049.99,7750.0,8043.82,364560.06474400,2019/09/30 01:59,178726.65304600,"(282.09, 39.65, Last Quarter)","(18.59, 2.79, New Moon)"


In [47]:
# create new columns based on tuple values
df[['Open Moon Degrees', 'Open Moon Percent', 'Open Moon phase']] = df['Open Moon'].apply(lambda x: pd.Series(x))

# drop original column
df.drop('Open Moon', axis=1, inplace=True)

# create new columns based on tuple values
df[['Close Moon Degrees', 'Close Moon Percent', 'Close Moon phase']] = df['Close Moon'].apply(lambda x: pd.Series(x))

# drop original column
df.drop('Close Moon', axis=1, inplace=True)

df.head()

,Open time,Open,High,Low,Close,Volume,Close time,Taker buy base asset volume,Open Moon Degrees,Open Moon Percent,Open Moon phase,Close Moon Degrees,Close Moon Percent,Close Moon phase
0,2017/08/14 02:00,4261.48,4485.39,3850.00,4086.29,2843.43142600,2017/08/21 01:59,2240.24924000,257.36,61.02,Waning Gibbous,350.94,0.63,Waning Crescent
1,2017/08/21 02:00,4069.13,4453.91,3400.00,4310.01,4599.39662900,2017/08/28 01:59,1755.53026500,350.95,0.63,Waning Crescent,76.09,38.15,Waxing Crescent
2,2017/08/28 02:00,4310.01,4939.19,4124.54,4509.08,4753.84337600,2017/09/04 01:59,1371.03442800,76.10,38.16,Waxing Crescent,153.70,94.84,Waxing Gibbous
3,2017/09/04 02:00,4505.00,4788.59,3603.00,4130.37,6382.78774500,2017/09/11 01:59,2821.42273900,153.70,94.84,Waxing Gibbous,241.14,74.13,Full Moon
4,2017/09/11 02:00,4153.62,4394.59,2817.00,3699.99,8106.70512700,2017/09/18 01:59,3763.12845200,241.15,74.12,Full Moon,332.99,5.48,Waning Crescent


### Let's determine the trend of the week

Since we are using weekly candles: 
- a closing price higher than the opening price means the price went UP during the week.
- a opening price higher than the closing price means the price went DOWN during the week.
- A new column "Change" represents with 1 that the prices went up and -1 that the prices went down.

In [48]:
up_or_down = lambda x: 1 if x >= 0 else -1 
df['Change'] = df['Open'].shift(1) - df['Close']
df['Change'] = df['Change'].apply(up_or_down)
df.head()

,Open time,Open,High,Low,Close,Volume,Close time,Taker buy base asset volume,Open Moon Degrees,Open Moon Percent,Open Moon phase,Close Moon Degrees,Close Moon Percent,Close Moon phase,Change
0,2017/08/14 02:00,4261.48,4485.39,3850.00,4086.29,2843.43142600,2017/08/21 01:59,2240.24924000,257.36,61.02,Waning Gibbous,350.94,0.63,Waning Crescent,-1
1,2017/08/21 02:00,4069.13,4453.91,3400.00,4310.01,4599.39662900,2017/08/28 01:59,1755.53026500,350.95,0.63,Waning Crescent,76.09,38.15,Waxing Crescent,-1
2,2017/08/28 02:00,4310.01,4939.19,4124.54,4509.08,4753.84337600,2017/09/04 01:59,1371.03442800,76.10,38.16,Waxing Crescent,153.70,94.84,Waxing Gibbous,-1
3,2017/09/04 02:00,4505.00,4788.59,3603.00,4130.37,6382.78774500,2017/09/11 01:59,2821.42273900,153.70,94.84,Waxing Gibbous,241.14,74.13,Full Moon,1
4,2017/09/11 02:00,4153.62,4394.59,2817.00,3699.99,8106.70512700,2017/09/18 01:59,3763.12845200,241.15,74.12,Full Moon,332.99,5.48,Waning Crescent,1


In [49]:
# Convert Open Moon phase and Close Moon phase to number
moons = {
    'New Moon': 0,
    'Waxing Crescent': 1,
    'First Quarter': 2,
    'Waxing Gibbous': 3,
    'Full Moon': 4,
    'Waning Gibbous': 5,
    'Last Quarter': 6,
    'Waning Crescent': 7,
}

df['Open Moon phase'] = df['Open Moon phase'].map(moons)
df['Close Moon phase'] = df['Close Moon phase'].map(moons)

df.head()

,Open time,Open,High,Low,Close,Volume,Close time,Taker buy base asset volume,Open Moon Degrees,Open Moon Percent,Open Moon phase,Close Moon Degrees,Close Moon Percent,Close Moon phase,Change
0,2017/08/14 02:00,4261.48,4485.39,3850.00,4086.29,2843.43142600,2017/08/21 01:59,2240.24924000,257.36,61.02,5,350.94,0.63,7,-1
1,2017/08/21 02:00,4069.13,4453.91,3400.00,4310.01,4599.39662900,2017/08/28 01:59,1755.53026500,350.95,0.63,7,76.09,38.15,1,-1
2,2017/08/28 02:00,4310.01,4939.19,4124.54,4509.08,4753.84337600,2017/09/04 01:59,1371.03442800,76.10,38.16,1,153.70,94.84,3,-1
3,2017/09/04 02:00,4505.00,4788.59,3603.00,4130.37,6382.78774500,2017/09/11 01:59,2821.42273900,153.70,94.84,3,241.14,74.13,4,1
4,2017/09/11 02:00,4153.62,4394.59,2817.00,3699.99,8106.70512700,2017/09/18 01:59,3763.12845200,241.15,74.12,4,332.99,5.48,7,1


In [50]:
# add previous 7 past weeks values to de dataframe
shifted_df = pd.DataFrame()
for i in range(1, 8):
    shifted_df[f'Volume_{i}'] = df['Volume'].shift(i)
    shifted_df[f'Taker buy base asset volume_{i}'] = df['Taker buy base asset volume'].shift(i)
    shifted_df[f'Open Moon Degrees_{i}'] = df['Open Moon Degrees'].shift(i)
    shifted_df[f'Open Moon Percent_{i}'] = df['Open Moon Percent'].shift(i)
    shifted_df[f'Open Moon phase_{i}'] = df['Open Moon phase'].shift(i)
    shifted_df[f'Close Moon Degrees_{i}'] = df['Close Moon Degrees'].shift(i)
    shifted_df[f'Close Moon Percent_{i}'] = df['Close Moon Percent'].shift(i)
    shifted_df[f'Close Moon phase_{i}'] = df['Close Moon phase'].shift(i)
    shifted_df[f'Change_{i}'] = df['Change'].shift(i)
# drop the first 7 rows since they contain NaN values
shifted_df = shifted_df.dropna()

# combine the original dataframe and the shifted dataframe
merged_df = pd.concat([df, shifted_df], axis=1)

In [51]:
merged_df = merged_df.dropna()
merged_df.head()

,Open time,Open,High,Low,Close,Volume,Close time,Taker buy base asset volume,Open Moon Degrees,Open Moon Percent,...,Change_6,Volume_7,Taker buy base asset volume_7,Open Moon Degrees_7,Open Moon Percent_7,Open Moon phase_7,Close Moon Degrees_7,Close Moon Percent_7,Close Moon phase_7,Change_7
7,2017/10/02 02:00,4400.00,4658.00,4110.00,4640.00,4708.27295600,2017/10/09 01:59,2077.80349000,134.31,84.99,...,-1.0,2843.43142600,2240.24924000,257.36,61.02,5.0,350.94,0.63,7.0,-1.0
8,2017/10/09 02:00,4640.00,5922.30,4550.00,5709.99,7913.92583700,2017/10/16 01:59,4280.12726900,224.19,85.78,...,-1.0,4599.39662900,1755.53026500,350.95,0.63,7.0,76.09,38.15,1.0,-1.0
9,2017/10/16 02:00,5710.00,6171.00,5037.95,5950.02,12157.76944700,2017/10/23 01:59,5501.91240000,315.31,14.53,...,1.0,4753.84337600,1371.03442800,76.10,38.16,1.0,153.70,94.84,3.0,-1.0
10,2017/10/23 02:00,5975.00,6189.88,5286.98,6169.98,13133.99457000,2017/10/30 00:59,5845.14325200,37.22,10.37,...,1.0,6382.78774500,2821.42273900,153.70,94.84,3.0,241.14,74.13,4.0,1.0
11,2017/10/30 01:00,6133.01,7590.25,6030.00,7345.01,11663.20964800,2017/11/06 00:59,5037.95081100,113.85,70.32,...,1.0,8106.70512700,3763.12845200,241.15,74.12,4.0,332.99,5.48,7.0,1.0


#### Just for reference

target:

- Change

features to consider:

- Volume
- Taker buy base asset volume
- Open Moon Degrees	
- Open Moon Percent
- Open Moon phase
- Close Moon Degrees
- Close Moon Percent
- Close Moon phase

and the "i" features from 1 to 7:

- Volume[i]
- Taker buy base asset volume[i]
- Open Moon Degrees[i]	
- Open Moon Percent[i]
- Open Moon phase[i]	
- Close Moon Degrees[i]	
- Close Moon Percent[i]	
- Close Moon phase[i]	
- Change[i]


## Now lets try to use this DataFrame to predict if the BTC will go up or will go down on next week

### Lets do a linear regression, Comparing Lasso and Ridge

In [58]:
dt_features = merged_df.drop(['Open time','Close time','Change'], axis=1)
dt_target = merged_df['Change']

# dt_features = StandardScaler().fit_transform(dt_features)

X_train, X_test, y_train, y_test = train_test_split(dt_features, dt_target, test_size=0.25) # , random_state=42

# entrenamos y predecimos con regresion lineal
modelLinear = LinearRegression().fit(X_train, y_train)
y_predict_linear =  modelLinear.predict(X_test)

# entrenamos y predecimos con lasso
# alpha representa el valor de penalización normalmente conocido como lambda
modelLasso = Lasso(alpha=0.03, max_iter=100000).fit(X_train, y_train)
y_predict_lasso = modelLasso.predict(X_test)

# lo mismo pero con el modelo ridge #alpha en este caso tiene el valor por defecto
modelRidge = Ridge(alpha=1, max_iter=100000).fit(X_train, y_train)
y_predict_ridge = modelRidge.predict(X_test)

# evaluar las predicciones
linear_loss = mean_squared_error(y_test, y_predict_linear)
print("Linear Loss:", linear_loss)

# se calcula la perdida por error medio cuadratico REVISA LA SECCION DE METRICAS DE SCIKITLEARN
lasso_loss = mean_squared_error(y_test, y_predict_lasso)
print("Lasso Loss: ", lasso_loss)

# 
ridge_loss = mean_squared_error(y_test, y_predict_ridge)
print("Ridge Loss: ", ridge_loss)

# 
print("="*32)
print("Coef LASSO")
print(modelLasso.coef_)

#
print("="*32)
print("Coef RIDGE")
print(modelRidge.coef_)


Linear Loss: 0.7809966077202805
Lasso Loss:  0.6702031376352484
Ridge Loss:  0.7090476247055187
Coef LASSO
[ 1.58022565e-04  2.50238913e-05 -2.81367316e-05 -1.59892144e-04
  9.20278868e-06 -2.00317648e-05 -0.00000000e+00 -0.00000000e+00
 -0.00000000e+00  1.91168308e-03  1.13203322e-02  0.00000000e+00
  1.94345247e-05 -3.96223588e-05  0.00000000e+00 -9.75211397e-03
  0.00000000e+00 -5.79694057e-05 -2.62927056e-03 -4.50730607e-02
  3.75191206e-01 -1.50599256e-05  3.03373981e-05  5.07344032e-04
  0.00000000e+00 -0.00000000e+00  1.14717528e-04 -5.83123063e-03
  0.00000000e+00  0.00000000e+00 -1.38056437e-05  3.01226758e-05
 -6.37820906e-04  0.00000000e+00  1.53710566e-02  0.00000000e+00
  0.00000000e+00 -0.00000000e+00  5.95010629e-02  3.31589616e-06
 -7.59043072e-06 -7.46038671e-05 -0.00000000e+00  0.00000000e+00
 -0.00000000e+00  0.00000000e+00  4.60847517e-03 -7.28583260e-02
  4.84328990e-06 -1.05416723e-05 -7.69588788e-06  2.44064253e-02
  4.69258150e-02 -0.00000000e+00 -0.00000000e+00

c:\Users\FelixGarcia\Develop\Python\MoonBTC\venv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.466e+01, tolerance: 1.927e-02
  model = cd_fast.enet_coordinate_descent(
